In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import Markdown as md
from datetime import date, datetime
%matplotlib inline
#sns.set(rc={"figure.figsize":(20, 20)})

from pandas import DataFrame
from typing import List
%matplotlib inline
import numpy as np

In [2]:
# COLUMNS
FECHAI = 'Fecha-I'
FECHAO = 'Fecha-O'
VLOO = 'Vlo-O'
ORIO = 'Ori-O'
DESO = 'Des-O'
EMPO = 'Emp-O'
VLOI = 'Vlo-I'
DIA = 'DIA'
MES = 'MES'
ANIO = 'AÑO'
DIANOM = 'DIANOM'
TIPOVUELO = 'TIPOVUELO'
OPERA = 'OPERA'
SIGLADES = 'SIGLADES'
COHORT = 'cohort'
# 
TEMPORADA_ALTA = 'temporada_alta'
DIF_MIN = 'dif_min'
ATRASO_15 = 'atraso_15'
PERIODO_DIA = 'periodo_dia'
MORNING = 'mañana'
AFTERNOON = 'tarde'
NIGHT = 'noche'

COL_TEMPORAL = 'col_tmp'

DATE = 'Date'
TIME = 'time'
HOUR = 'hour'
MINUTE = 'minute'


COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, TIPOVUELO, OPERA, SIGLADES, 
                TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA]
COLUM_SYNTHETIC_FEATURES = [TEMPORADA_ALTA, DIF_MIN, ATRASO_15, PERIODO_DIA]




STR_WINTER_START = '2017-12-15'
STR_WINTER_END = '2017-12-31'
STR_WINTER_BIS_START = '2017-01-01'
STR_WINTER_BIS_END = '2017-03-03'
STR_JULY_START = '2017-07-15'
STR_JULY_END = '2017-07-31'
STR_SEPT_START = '2017-09-11'
STR_SEPT_END = '2017-09-30'


STR_MANANA_START= '5:00'
STR_MANANA_END = '11:59'
STR_TARDE_START = '12:00'
STR_TARDE_END = '18:59'

PARAM_GRID = {
    'n_estimators': [10, 20, 50, 10, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [4, 8, 16, 32, 64, 128, 256, 512],
    'criterion': ['gini', 'entropy'],
    'n_jobs': [2]
}


columns = [TEMPORADA_ALTA, DIF_MIN, ATRASO_15, PERIODO_DIA]
def convert_str_to_date(str_date: str) -> datetime:
    """convert string to date format '%Y-%m-%d'

    Args:
        str_date (str): string ex: '2017-01-01'

    Returns:
        datetime: string with datetime type
    """
    return datetime.strptime(str_date, '%Y-%m-%d').date()

def convert_str_to_time(str_date: str) -> datetime:
    """convert string to time format '%H:%M'

    Args:
        str_date (str): string '23:59'

    Returns:
        datetime: string with datetime type
    """
    return datetime.strptime(str_date, '%H:%M').time()

def add_temporada_alta_flag(df: DataFrame, dates: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    df[TEMPORADA_ALTA] = 0
    for date in dates:
        df.loc[(df[DATE] >= date[0]) & (df[DATE]<=date[1]), TEMPORADA_ALTA] = 1
    return df.copy()
def add_periodo_dia_flag(df: DataFrame, times: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    # default
    df[PERIODO_DIA] = NIGHT
    df.loc[(df[TIME] >= times[0][0]) & (df[TIME]<=times[0][1]), PERIODO_DIA] = MORNING
    df.loc[(df[TIME] >= times[1][0]) & (df[TIME]<=times[1][1]), PERIODO_DIA] = AFTERNOON
    return df.copy()

In [3]:
def generate_temporada_alta_set()-> list:
    """
    This function generate arrays to generate temporada_alta elements, source from constant
    dates ahre the following, 
        15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep,
    output: array of element, with the following 
    """
    str_winter_start = convert_str_to_date(STR_WINTER_START)
    str_winter_end = convert_str_to_date(STR_WINTER_END)
    str_winter_bis_start = convert_str_to_date(STR_WINTER_BIS_START)
    str_winter_bis_end = convert_str_to_date(STR_WINTER_BIS_END)
    str_July_start = convert_str_to_date(STR_JULY_START)
    str_July_end = convert_str_to_date(STR_JULY_END)
    str_Sept_start = convert_str_to_date(STR_SEPT_START)
    str_Sept_end = convert_str_to_date(STR_SEPT_END)

    dates = [[str_winter_start, str_winter_end],
             [str_winter_bis_start, str_winter_bis_end],
             [str_July_start, str_July_end],
             [str_Sept_start, str_Sept_end]]
    return dates

def generate_day_section():
    """
    split a day iin three part, morning, afternoon and night
    """
    str_mañana_start = convert_str_to_time(STR_MANANA_START)
    str_mañana_end = convert_str_to_time(STR_MANANA_END)
    str_tarde_start = convert_str_to_time(STR_TARDE_START)
    str_tarde_end = convert_str_to_time(STR_TARDE_END)
    times = [[str_mañana_start, str_mañana_end],
             [str_tarde_start, str_tarde_end],
            ]
    return times

def split_date(df:DataFrame, column_name:str)-> DataFrame:
    """_summary_

    Args:
        df (DataFrame): _description_
        column_name (str): _description_

    Returns:
        DataFrame: _description_
    """
    df[column_name] = pd.to_datetime(df[column_name])
    df[DATE] = df[column_name].dt.date
    df[TIME] = df[column_name].dt.time
    return df.copy()

def add_temporada_alta_flag(df: DataFrame, dates: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    df[TEMPORADA_ALTA] = 0
    for date in dates:
        df.loc[(df[DATE] >= date[0]) & (df[DATE]<=date[1]), TEMPORADA_ALTA] = 1
    return df.copy()

def add_periodo_dia_flag(df: DataFrame, times: List) -> DataFrame:
    """Add temporada alta flag according to array of dates

    Args:
        df (DataFrame): _description_
        dates (List): _description_

    Returns:
        DataFrame: _description_
    """
    # default
    df[PERIODO_DIA] = NIGHT
    df.loc[(df[TIME] >= times[0][0]) & (df[TIME]<=times[0][1]), PERIODO_DIA] = MORNING
    df.loc[(df[TIME] >= times[1][0]) & (df[TIME]<=times[1][1]), PERIODO_DIA] = AFTERNOON
    return df.copy()

def add_dif_min_and_atraso_15(df:DataFrame) ->DataFrame:
    df[DIF_MIN] = df[FECHAO]-df[FECHAI]
    df[DIF_MIN] = df[DIF_MIN].dt.total_seconds()//60
    df[ATRASO_15] = 0
    df.loc[df[DIF_MIN]>15, ATRASO_15] = 1
    return df



In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

In [30]:
def delete_operational_columns(df:DataFrame)-> DataFrame:
    del df[FECHAO]
    del df[VLOO]
    del df[ORIO]
    del df[DESO]
    del df[EMPO]
    return df

def filter_column(df:DataFrame, columns:str =  COLUM_FILTER)->DataFrame:
    return df[columns]


def previous_flight (df):
    df.sort_values(FECHAI, inplace = True)
    df["Early"] = df[DIF_MIN].shift(1)
    df['AnteriorEarly'] = 0
    df.loc[df["Early"]<0, 'AnteriorEarly'] = 1
    df.loc[df['AnteriorEarly'].isna(), 'AnteriorEarly'] = 0
    df["Delay"] = df[DIF_MIN].shift(1)
    df['AnteriorDelay'] = 0
    df.loc[df["Delay"]>0, 'AnteriorDelay'] = 1
    del df['Delay']
    del df['Early']
    return df

def number_of_flights_before (df):

    df.sort_values(FECHAI, inplace = True)
    dft = df.groupby([FECHAI])[FECHAI].count().rename("col_tmp")
    dft = dft.reset_index()
    dft["number_of_flights_before"] = dft["col_tmp"].shift(1)
    del dft['col_tmp']
    df = pd.merge(df,dft)
    df.loc[df['number_of_flights_before'].isna(), 'number_of_flights_before'] = 0

    return df


def number_of_flights_same_opera(df):
    dft = df.groupby([FECHAI, 'OPERA'])[FECHAI].count().rename("number_of_flights_same_opera")
    dft = dft.reset_index()
    dft['number_of_flights_same_opera'] = dft['number_of_flights_same_opera'] -1 
    df = pd.merge(df,dft)
    df.loc[df['number_of_flights_same_opera'].isna(), 'number_of_flights_same_opera'] = 0
    return df

def number_of_flights_same_dest(df):
    dft = df.groupby([FECHAI, 'SIGLADES'])[FECHAI].count().rename("number_of_flights_same_dest")
    dft = dft.reset_index()
    dft['number_of_flights_same_dest'] = dft['number_of_flights_same_dest'] -1 
    df = pd.merge(df,dft)
    df.loc[df['number_of_flights_same_dest'].isna(), 'number_of_flights_same_dest'] = 0
    return df

def one_hot_encoder(df:DataFrame)-> DataFrame:
    
    df = pd.get_dummies(df, columns = [OPERA])
    df = pd.get_dummies(df, columns = [SIGLADES])
    df = pd.get_dummies(df, columns = [TEMPORADA_ALTA]) 
    df = pd.get_dummies(df, columns = [PERIODO_DIA]) 
    df = pd.get_dummies(df, columns = [VLOI])
    df = pd.get_dummies(df, columns = [DIANOM])
    return df 


def to_bool_tipo_vuelo(df:DataFrame)-> DataFrame:
    df[f"{TIPOVUELO}_"] = df[TIPOVUELO]
    df.loc[df[f"{TIPOVUELO}_"]== 'I', TIPOVUELO] = 1
    df.loc[df[f"{TIPOVUELO}_"]== 'N', TIPOVUELO] = 0
    del df[f"{TIPOVUELO}_"]
    return df

def fit_grid_model(
    rfc: RandomForestClassifier,
    param_grid,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
):  # pragma: no cover
    """GridSeach

    Args:
        rfc (RandomForestClassifier): ML Model
        param_grid (dict): JSON parameters
        X_train (pd.DataFrame): train df
        y_train (pd.DataFrame): goal df
        model_type (str): model name

    Returns:
        [type]: model trained
    """
    model = GridSearchCV(
        estimator=rfc,
        param_grid=param_grid,
        cv=3,
    )
    model.fit(X_train, y_train)
    return model

In [6]:
dates = generate_temporada_alta_set()
times = generate_day_section()


In [7]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(filter_column)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

no sirven
* Ori-I y Pri-o un sólo lugar de origen
* Des-I y SIGLADES son lo mismo
- OPERA == Emp-I

modificadores
- Vlo-I != Vlo-O, es diferente hubo atrasosi el número de vuelo cambio o el origen cambio, significa un retraso? Des-I
- Des-I != Des-O, es diferente, atraso
# no se usaran para evitar sesgos
- Fecha-O


In [8]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [10]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
 

In [11]:
# Select 4 new classifiers
clfs = {"Logistic Regression": LogisticRegression(solver = 'saga',  random_state=0,  max_iter=99 ),
        "Support Vector rbf": SVC(kernel='rbf'),
        "Support Vector sigmoid": SVC(kernel='sigmoid'),
        "KNN":KNeighborsClassifier(n_neighbors=15),
        "Decision Tree": DecisionTreeClassifier()
       }

In [12]:
from sklearn.inspection import permutation_importance

def feature_importance(clf, X, y, top_limit=None):

    # Retrieve the Bunch object after 50 repeats
    # n_repeats is the number of times that each feature was permuted to compute the final score
    bunch = permutation_importance(clf, X, y,
                                 n_repeats=50, random_state=42)

    # Average feature importance
    imp_means = bunch.importances_mean

    # List that contains the index of each feature in descending order of importance
    ordered_imp_means_args = np.argsort(imp_means)[::-1]

    # If no limit print all features
    if top_limit is None:
        top_limit = len(ordered_imp_means_args)

    # Print relevant information
    for i, _ in zip(ordered_imp_means_args, range(top_limit)):
        name = data.feature_names[i]
        imp_score = imp_means[i]
        imp_std = bunch.importances_std[i]
        print(f"Feature {name} with index {i} has an average importance score of {imp_score:.3f} +/- {imp_std:.3f}\n")
        # Compute feature importance on the test set given a classifier

def fit_compute_importance(clf, k = 4):
    #kf = KFold(n_splits=k, random_state=None)
    #result = cross_val_score(clf , X_train, y_train, cv = kf)
    #print(f"cross-validation Avg accuracy: {result.mean()}")
    
    clf.fit(X_train, y_train)
    print(f"Mean accuracy score on the test set: {clf.score(X_test, y_test)*100:.2f}%\n")
    # print(f"📏 Mean accuracy score on the test set: {clf.score(X_test, y_test)*100:.2f}%\n")
    # print("🔝 Top 4 features when using the test set:\n")
    #feature_importance(clf, X_test, y_test, top_limit=4)

In [13]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.61%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.61%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.61%

KNN classifier

Mean accuracy score on the test set: 81.50%

Decision Tree classifier

Mean accuracy score on the test set: 74.97%



In [14]:
rfc_st = RandomForestClassifier(random_state=42)
model = fit_grid_model(rfc_st, PARAM_GRID, X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8192797580866856

# 1 adding new feature previous_flight

In [15]:
COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, 
                TIPOVUELO, OPERA, SIGLADES, TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA, 'AnteriorDelay', 'AnteriorEarly']


In [24]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(previous_flight)
        .pipe(filter_column, COLUM_FILTER)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

In [25]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [26]:
print(columns)

['DIA', 'MES', 'AÑO', 'TIPOVUELO', 'AnteriorDelay', 'AnteriorEarly', 'OPERA_Aerolineas Argentinas', 'OPERA_Aeromexico', 'OPERA_Air Canada', 'OPERA_Air France', 'OPERA_Alitalia', 'OPERA_American Airlines', 'OPERA_Austral', 'OPERA_Avianca', 'OPERA_British Airways', 'OPERA_Copa Air', 'OPERA_Delta Air', 'OPERA_Gol Trans', 'OPERA_Grupo LATAM', 'OPERA_Iberia', 'OPERA_JetSmart SPA', 'OPERA_K.L.M.', 'OPERA_Lacsa', 'OPERA_Latin American Wings', 'OPERA_Oceanair Linhas Aereas', 'OPERA_Plus Ultra Lineas Aereas', 'OPERA_Qantas Airways', 'OPERA_Sky Airline', 'OPERA_United Airlines', 'SIGLADES_Antofagasta', 'SIGLADES_Arica', 'SIGLADES_Asuncion', 'SIGLADES_Atlanta', 'SIGLADES_Auckland N.Z.', 'SIGLADES_Balmaceda', 'SIGLADES_Bariloche', 'SIGLADES_Bogota', 'SIGLADES_Buenos Aires', 'SIGLADES_Calama', 'SIGLADES_Cancun', 'SIGLADES_Castro (Chiloe)', 'SIGLADES_Cataratas Iguacu', 'SIGLADES_Ciudad de Mexico', 'SIGLADES_Ciudad de Panama', 'SIGLADES_Cochabamba', 'SIGLADES_Concepcion', 'SIGLADES_Copiapo', 'SIGLADE

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [28]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.43%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.43%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.43%

KNN classifier

Mean accuracy score on the test set: 81.44%

Decision Tree classifier

Mean accuracy score on the test set: 74.53%



In [29]:
rfc_st = RandomForestClassifier(random_state=42)
model = fit_grid_model(rfc_st, PARAM_GRID, X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8161642078255292

# 2  number_of_flights_before

In [31]:
COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, 
                TIPOVUELO, OPERA, SIGLADES, TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA, 'number_of_flights_before']


In [32]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(number_of_flights_before)
        .pipe(filter_column, COLUM_FILTER)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

In [33]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [35]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.43%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.43%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.43%

KNN classifier

Mean accuracy score on the test set: 81.41%

Decision Tree classifier

Mean accuracy score on the test set: 75.02%



In [47]:
COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, 
                TIPOVUELO, OPERA, SIGLADES, TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA, 'number_of_flights_same_opera']


# 3  number_of_flights_same_opera

In [48]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(number_of_flights_same_opera)
        .pipe(filter_column, COLUM_FILTER)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

In [49]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [50]:
columns

['DIA',
 'MES',
 'AÑO',
 'TIPOVUELO',
 'number_of_flights_same_opera',
 'OPERA_Aerolineas Argentinas',
 'OPERA_Aeromexico',
 'OPERA_Air Canada',
 'OPERA_Air France',
 'OPERA_Alitalia',
 'OPERA_American Airlines',
 'OPERA_Austral',
 'OPERA_Avianca',
 'OPERA_British Airways',
 'OPERA_Copa Air',
 'OPERA_Delta Air',
 'OPERA_Gol Trans',
 'OPERA_Grupo LATAM',
 'OPERA_Iberia',
 'OPERA_JetSmart SPA',
 'OPERA_K.L.M.',
 'OPERA_Lacsa',
 'OPERA_Latin American Wings',
 'OPERA_Oceanair Linhas Aereas',
 'OPERA_Plus Ultra Lineas Aereas',
 'OPERA_Qantas Airways',
 'OPERA_Sky Airline',
 'OPERA_United Airlines',
 'SIGLADES_Antofagasta',
 'SIGLADES_Arica',
 'SIGLADES_Asuncion',
 'SIGLADES_Atlanta',
 'SIGLADES_Auckland N.Z.',
 'SIGLADES_Balmaceda',
 'SIGLADES_Bariloche',
 'SIGLADES_Bogota',
 'SIGLADES_Buenos Aires',
 'SIGLADES_Calama',
 'SIGLADES_Cancun',
 'SIGLADES_Castro (Chiloe)',
 'SIGLADES_Cataratas Iguacu',
 'SIGLADES_Ciudad de Mexico',
 'SIGLADES_Ciudad de Panama',
 'SIGLADES_Cochabamba',
 'SIGLADES

In [51]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [52]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.58%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.58%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.58%

KNN classifier

Mean accuracy score on the test set: 81.63%

Decision Tree classifier

Mean accuracy score on the test set: 74.22%



# 4  number_of_flights_same_dest

In [41]:
COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, 
                TIPOVUELO, OPERA, SIGLADES, TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA, 'number_of_flights_same_dest']


In [42]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(previous_flight)
        .pipe(number_of_flights_before)
        .pipe(number_of_flights_same_opera)
        .pipe(number_of_flights_same_dest)
        .pipe(filter_column, COLUM_FILTER)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

In [43]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [45]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.59%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.59%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.59%

KNN classifier

Mean accuracy score on the test set: 81.51%

Decision Tree classifier

Mean accuracy score on the test set: 74.91%



# All features

In [53]:
COLUM_FILTER = [FECHAI,VLOI, DIA, MES, ANIO, DIANOM, 
                TIPOVUELO, OPERA, SIGLADES, TEMPORADA_ALTA, ATRASO_15, PERIODO_DIA,
                 'AnteriorDelay', 'AnteriorEarly', 'number_of_flights_same_opera',
                'number_of_flights_before', 'number_of_flights_same_dest']


In [54]:
df = pd.read_csv("../sources/raw/dataset_SCL.csv", dtype={'Vlo-O':str, 'Vlo-I':str})
df[FECHAO] = pd.to_datetime(df[FECHAO])
df = (df.pipe(split_date, FECHAI)
        .pipe(add_temporada_alta_flag, dates)
        .pipe(add_periodo_dia_flag, times)
        #.pipe(apply_cohort)
        .pipe(add_dif_min_and_atraso_15)
        .pipe(delete_operational_columns)
        .pipe(previous_flight)
        .pipe(number_of_flights_before)
        .pipe(number_of_flights_same_opera)
        .pipe(number_of_flights_same_dest)
        .pipe(filter_column, COLUM_FILTER)
        .pipe(to_bool_tipo_vuelo)
        .pipe(one_hot_encoder) # generate one_hot_encoder
     )

In [55]:
columns = df.columns
columns = columns.tolist()
columns.remove(FECHAI)
df = df[columns]
columns.remove(ATRASO_15)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
        df[columns],
        df[ATRASO_15],
        test_size=.8,
        random_state=42,
    )

In [57]:
# Print results
for name, clf in clfs.items():
    print("====="*20)
    print(f"{name} classifier\n")
    fit_compute_importance(clf)

Logistic Regression classifier



/opt/anaconda3/envs/ag_gk/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Mean accuracy score on the test set: 81.44%

Support Vector rbf classifier

Mean accuracy score on the test set: 81.44%

Support Vector sigmoid classifier

Mean accuracy score on the test set: 81.44%

KNN classifier

Mean accuracy score on the test set: 81.50%

Decision Tree classifier

Mean accuracy score on the test set: 74.15%



In [58]:
rfc_st = RandomForestClassifier(random_state=42)
model = fit_grid_model(rfc_st, PARAM_GRID, X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8187116283331806

In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)


0.8142948776688353